# Wizualizacja poissonowskiego strumienia zgłoszeń

## Piotr Serafin 132821

## Rozkład Poissona[1]

Dyskretny rozkład prawdopodobieństwa, wyrażający prawdopodobieństwo szeregu wydarzeń mających miejsce w określonym czasie, gdy te wydarzenia występują ze znaną średnią częstotliwością i w sposób niezależny od czasu jaki upłynął od ostatniego zajścia takiego zdarzenia. Rozkład Poissona można również stosować w odniesieniu do liczby zdarzeń w innych określonych przedziałach, takich jak odległość, powierzchnia lub objętość. 

Jeśli oczekiwaną liczbą zdarzeń w tym przedziale jest λ, to prawdopodobieństwo, że jest dokładnie k wystąpień jest równe:

$$f(k,\lambda) = \frac{\lambda^k e^{-\lambda}}{k !}$$

gdzie:

* $\lambda$ jest dodatnią liczbą rzeczywistą, równą oczekiwanej liczbie zdarzeń w danym przedziale czasu
* k jest liczbą wystąpień zdarzenia

Rozkład Poissona powstaje w związku z procesami Poissona. Ma on zastosowanie do różnych zjawisk dyskretnych właściwości, gdy prawdopodobieństwo wystąpienia zjawiska jest stałe w czasie lub przestrzeni. Zwykłym zastosowaniem rozkładu Poissona jest prognozowanie liczby zdarzeń w danym czasie.

In [ ]:
# Import bibliotek
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson

%matplotlib inline

In [ ]:
# Funkcja zwracająca tablcę prawdopodobieństw dla zadanych parametrów k
# Suma prawdopodobieństw wynosi 1 (Funkcja masy prawdopodobieństw)
def rozkladPoissonaPmf(K, mu, t = 1):
    return np.array([rozkladPoissona(k, mu, t) for k in np.nditer(K)])

# Funkcja wyliczająca prawdopodobieństwo naplywu k zgloszen w przedziale czasu t 
# dla wartości oczekiwanej mu
def rozkladPoissona(k, mu, t = 1):
    return math.pow(mu * t, k) * math.exp(-mu * t) / math.factorial(k)


In [ ]:
# Tablica parametrów k 0..20
K = np.arange(0, 20, 1)[:, None]

# Tablica dla różnych wartości parametru lambda 0..10
mu_array = np.arange(0, 10, 1)[:, None]

# Macierz prawdopodobieństw dla wszystkich kombinacji k, mu
pmf_array = np.array([rozkladPoissonaPmf(K, mu) for mu in mu_array]).reshape(mu_array.shape[0], K.shape[0])

# Alternatywnie funkcja pmf z biblioteki scipy: 
# pmf_array = np.array([poisson(mu).pmf(K) for mu in mu_array]).reshape(mu_array.shape[0], K.shape[0])

# Tablica przykładowych wartości lambdy
mu_plot_array = [0,1,3,5,8]

# Zmiana domyślnej wielkości wykresu - aspect ratio 5:3
plt.rcParams['figure.figsize'] = (15, 9)

for mu in mu_plot_array:
    plt.plot(K, pmf_array[mu, :], '-o', label='$\lambda = {}$'.format(int(mu_array[mu])))

# Opis wykresu
plt.title('Funkcja rozkładu Poissona dla $\lambda$ = {}'.format(mu_plot_array))
plt.ylabel('$f(k,\lambda)$')
plt.xlabel('$k$')
plt.legend()
plt.show()

## Strumień zgłoszeń[2]

Strumień zgłoszeń opisany jest w pełni przez zbiór reguł określających jednoznacznie proces napływu zgłoszeń do systemu obsługi w określonym przedziale czasu (statystyczny opis procesu przybywania zgłoszeń do systemu obsługi).

### Klasyfikacja strumieni zgłoszeń

- regularny jeśli tworzące go zdarzenia pojawiają się w zdeterminowanych przedziałach czasu
- stochastyczny jeśli tworzące go zdarzenia pojawiają się losowo
- jednorodny – charakteryzuje go zawsze jedna cecha
- niejednorodny – każde zgłoszenie ma co najmniej dwie cechy (jedną z nich jest zawsze czas napływu zgłoszenia)

Strumień ruchu opisany rozkładem Poissona jest przypadkiem strumienia prostego.

### Cechy strumienia prostego:

- Stacjonarność – prawdopodobieństwo pojawienia się pewnej liczby zgłoszeń w przedziale czasu zależy od długości tego przedziału, a nie jego położenia na osi czasu
- Brak pamięci - dla dowolnych rozłącznych przedziałów czasu liczba zgłoszeń zachodzących w jednym z nich nie zależy od liczby zgłoszeń zachodzących w pozostałych przedziałach
- Pojedynczość – dwa lub więcej zgłoszeń nie mogą się pojawić w tym samym czasie

Rozkład Poissona bardzo dobrze opisuje jeden strumień zgłoszeń generowany przez bardzo dużą liczbę źródeł ruchu, teoretycznie nieskończenie dużą

Prawdopodobieństwo napływu k zgłoszeń w przedziale czasu t przy intensywności zgłoszeń $\lambda$:

$$P(t) = \frac{(\lambda t)^k e^{-\lambda t}} {k !}$$


## Generowanie strumienia zgłoszeń

Strumień zdarzeń o rozkładzie Poissona charakteryzuje się wykładniczym rozkładem długości odstępów czasu między zdarzeniami. Jeżeli założymy, że 2400 połączeń wykonywanych jest w ciągu dnia, to średnio $\frac{2400}{24} = 100$ połączenień wykonywanych jest w ciągu godziny. Z tego wynika, że co $\lambda = 72$ sekundy napływa zgłoszenie. Mając te dane, możemy zadać sobie pytanie: 

Jakie jest prawdopodobieństwo, żę w ciągu następnych 20 sekund pojawi się zgłoszenie?

Na to pytanie odpowiada dystrybuanta rozkładu wykładniczego przedstawiona poniżej.

In [ ]:
def dystrybuantaRozkladuWykladniczego(mu, t = 1):
    return (1 - Math.exp(-mu * t))

def funkcjaOdwrotnaDystrybuanyRozkladuWykladniczego(probability, mu):
    return (-Math.log(1 - probability)/mu)


In [ ]:
# Dystrybuanta rozkładu wykładniczego
T = np.arange(0, 360, 1)[:, None]
cdf_array = np.array([poisson.cdf(K, mu) for mu in mu_array]).reshape(mu_array.shape[0], K.shape[0])

for mu in mu_plot_array:
    plt.plot(K, cdf_array[mu, :], '-o', label='$\lambda = {}$'.format(int(mu_array[mu])))

plt.title('Dystrybuanta rozkładu wykładniczego dla $\lambda$ = {}'.format(mu_plot_array))
plt.ylabel('Prawdopodobieństwo')
plt.xlabel('Czas')
plt.legend()
plt.show()

## Generownie czasu zgłoszeń

Następnym krokiem w symulacji jest wygenerowanie czasu pojawienia się zgłoszenia w systemie. Czas ten powinien zgadzać się z rozkładem wykładniczym długości odstępów czasu między zdarzeniami. Aby to zrobić, użyjemy generatora liczb pseudolosowych między 0 i 1. Należy znaleźć funkcję odwrotną do funkcji dystrybuanty rozkładu wykładniczego:


$$F(x) = 1 - e^{-\lambda x}$$
$$e^{-\lambda x} = 1 - F(x)$$
$$-\lambda x = \ln (1-F(x))$$
$$x = \frac{-\ln(1-F(x))}{\lambda}$$

$$nastepnyCzas = \frac{-\ln U}{\lambda}$$

Gdzie U to liczba pseudolosowa z zakresu $[0,1)$

### Prawdopodobieństwo wystąpienia k żądań w odcinku czasu [0, t]

Ostatnim krokiem symulacji jest wyliczenie prawdopodobieństwa napływu k zgłoszeń w wygenerowanych przedziałach.
Korzystamy z ze wzoru definiującego rozkład Poissona. Prawdopodobieństwo napływu k zgłoszeń w przedziale czasu t przy intensywności zgłoszeń

$$P(t) = \frac{(\lambda t)^k e^{-\lambda t}} {k !}$$



## Bibliografia

[1]: Rozkład Poissona. (2018, listopad 19). Wikipedia, wolna encyklopedia. Dostęp 16:20, listopad 23, 2018,  
Dostępny w Internecie: http://pl.wikipedia.org/w/index.php?title=Rozk%C5%82ad_Poissona&oldid=55076364 

[2]: Klink, J. (2011). Inżynieria ruchu (telekomunikacyjnego) [Slajdy PDF].  
Dostępny w Internecie: https://pst.pwr.edu.pl/moodle/course/view.php?id=82

[3]: Kaliszan, A. Głąbowski, M. (2006). Symulator wiązki pełnodostępnej obsługującej zintegrowane nie-poissonowskie strumienie zgłoszeń [PDF]. Dostępny w Internecie: http://www.pwt.et.put.poznan.pl/srv/papers/PWT%202006_3658.pdf